In [1]:
!pip install pandas openai



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install xlrd


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from openai import OpenAI, types

client = OpenAI() 
client.api_key ="sk-test"
# Your code below:
response = client.chat.completions.create( 
  model="gpt-3.5-turbo", 
  messages=[ 
    { 
      "role": "user", 
      "content": "Tell me how many seconds there are in a day in a Shakespearean tone?" 
    } 
  ]) 

print(response.choices[0].message.content) 

Methinks there be twenty-four score seconds in a single day, from dawn til dusk, from morn til eventide. Aye, each second fleeting by with swift and silent grace, marking the passage of time like whispers in the wind.


In [5]:
print(response.choices[0].message.content) 

Forsooth, in a single day doth pass
Twenty-four hours, a thousand moments fast
Yet in each hour, sixty seconds doth remain
Thus in a day, doth four and twenty times sixty reign
A total of fourteen hundred and forty, be sure
Seconds in a day, ever passing, ever pure.


In [6]:
# Import the necessary libraries and initialize the client
from openai import OpenAI, types

client = OpenAI()
client.api_key ="sk-test"

# Sample paper details for testing
paper_details = {
    "title": "Mobility Patterns and the Spread of Infectious Diseases: Insights from Public Datasets",
    "abstract": (
        "This study leverages publicly available mobility data to analyze the effects of human movement "
        "patterns on the spread of infectious diseases. Using large-scale datasets derived from urban transit "
        "and cellphone location data, we explore correlations between mobility trends and infection rates. "
        "Findings suggest that reduced movement is associated with slower disease transmission in major metropolitan areas."
    ),
    "public_availability": "Yes",  # Can be "Yes" or "No" depending on paper's access policy
    "dataset_info": "The datasets used are from public transit authorities and anonymized cellphone data aggregates."
}

# Construct the prompt with clear instructions.
prompt = f"""
You are an expert in epidemiology and data science. I need you to evaluate the following research paper based on the criteria below:

1. **Relevance**: On a scale from 0 to 100, how strongly is this paper focused on the topic of human movement analysis and its effect on infectious diseases?
2. **Public Data Usage**: Does the paper use publicly available mobility datasets? Answer with "Yes" or "No".
3. **Justification**: Provide a brief explanation of your evaluation.

Here is the paper information:

Title: {paper_details['title']}

Abstract: {paper_details['abstract']}

Public Availability: {paper_details['public_availability']}

Dataset Information: {paper_details['dataset_info']}

Please provide your response in the following JSON format:

{{
  "relevance_score": <number between 0 and 100>,
  "public_data": "<Yes or No>",
  "justification": "<your explanation>"
}}
"""

# Call the OpenAI chat completion API
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

# Print the output from the model
print(response.choices[0].message.content)


ImportError: cannot import name 'OpenAI' from 'openai' (c:\ProgramData\anaconda3\Lib\site-packages\openai\__init__.py)

In [7]:
import pandas as pd
import json
import time
from openai import OpenAI, types

# Initialize the OpenAI client with your API key
client = OpenAI()
client.api_key ="sk-test"

# Read the Excel file into a pandas DataFrame.
# Adjust the file path if necessary.
df = pd.read_excel("savedrecs.xls")

# Create an empty list to store evaluation results
evaluation_results = []

# Define a function that builds the prompt for a given paper (row)
def build_prompt(row):
    # Extract fields from the row
    # Adjust the column names if necessary to match your Excel file
    title = row.get("Article Title", "No Title Provided")
    abstract = row.get("Abstract", "No Abstract Provided")
    # Optional: You may include other details if available. For example:
    # source_title = row.get("Source Title", "No Source Provided")
    # For this example, we focus on the title and abstract.
    
    prompt = f"""
You are an expert in epidemiology and data science. I need you to evaluate the following research paper based on the criteria below:

1. **Relevance**: On a scale from 0 to 100, how strongly is this paper focused on the topic of human movement analysis and its effect on infectious diseases?
2. **Public Data Usage**: Does the paper use publicly available mobility datasets? Answer with "Yes" or "No".
3. **Justification**: Provide a brief explanation of your evaluation.

Here is the paper information:

Title: {title}

Abstract: {abstract}

Please provide your response in the following JSON format:

{{
  "relevance_score": <number between 0 and 100>,
  "public_data": "<Yes or No>",
  "justification": "<your explanation>"
}}
    """
    return prompt

# Iterate over each paper in the DataFrame
# For demonstration purposes, you might want to process only a subset of rows.
for index, row in df.iterrows():
    prompt_text = build_prompt(row)
    
    try:
        # Call the OpenAI Chat Completion API
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt_text}
            ]
        )
        
        # Get the message content from the response
        response_content = response.choices[0].message.content.strip()
        print(f"Response for record {index}:")
        print(response_content)
        
        # Attempt to parse the response as JSON
        try:
            response_data = json.loads(response_content)
        except json.JSONDecodeError:
            # If the response is not valid JSON, store the raw string
            response_data = {"error": "Invalid JSON response", "raw_response": response_content}
        
        # Append the result along with a reference to the original paper (for example, title)
        evaluation_results.append({
            "index": index,
            "title": row.get("Article Title", "No Title Provided"),
            "evaluation": response_data
        })
    
    except Exception as e:
        print(f"An error occurred processing record {index}: {e}")
        evaluation_results.append({
            "index": index,
            "title": row.get("Article Title", "No Title Provided"),
            "evaluation": {"error": str(e)}
        })
    
    # Optional: Pause for a short duration to respect API rate limits
    time.sleep(1)

# Convert the results into a DataFrame for further analysis or saving
results_df = pd.DataFrame(evaluation_results)

# Display the evaluation results
print(results_df)

# Optionally, you can save the results to a new Excel or CSV file:
results_df.to_excel("evaluation_results.xlsx", index=False)
# or
results_df.to_csv("evaluation_results.csv", index=False)


ImportError: cannot import name 'OpenAI' from 'openai' (c:\ProgramData\anaconda3\Lib\site-packages\openai\__init__.py)

In [4]:
from openai import OpenAI
client = OpenAI()
client.api_key ="sk-test"

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Code within itself,  \nLoops unfold, yet still return—  \nInfinite echoes.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [5]:
import random
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI()
client.api_key ="sk-test"

# List of random abstracts (you can replace these with actual abstracts)
abstracts = [
    "This study explores the relationship between urban planning and pedestrian behavior in metropolitan areas.",
    "The effects of global warming on marine ecosystems have been studied extensively over the past decade.",
    "Human mobility patterns during pandemics reveal significant shifts in travel behavior and work locations.",
    "Advancements in renewable energy have led to a significant reduction in carbon emissions globally.",
    "Analyzing migration trends over the past century highlights the socio-economic factors driving relocation."
]

# Choose a random abstract
random_abstract = random.choice(abstracts)

# Make a completion request to the OpenAI API
response = client.chat.completions.create(
    model="gpt-4o",  # You can replace this with the desired GPT model
    messages=[
        {"role": "system", "content": "You are an expert on human mobility and its related topics."},
        {"role": "user", "content": f"Rate the relevance of the following abstract to the topic of human mobility on a scale of 1 to 10. Provide a brief explanation for your score.\n\nAbstract: {random_abstract}"}
    ]
)

# Extract the model's response
relevance_score = response.choices[0].message.content

# Print the abstract and the relevance score
print(f"Abstract: {random_abstract}\n")
print(f"Relevance Score and Explanation: {relevance_score}")


Abstract: This study explores the relationship between urban planning and pedestrian behavior in metropolitan areas.

Relevance Score and Explanation: I would rate the relevance of this abstract to the topic of human mobility as an 8 out of 10. 

Explanation: Human mobility refers to the movement of people across various spaces, and pedestrian behavior is a fundamental aspect of this. The abstract discusses the connection between urban planning and how pedestrians move and behave in metropolitan areas, which directly relates to understanding and influencing human mobility patterns. Urban planning can significantly impact the efficiency, safety, and accessibility of pedestrian pathways, all of which are crucial elements of human mobility in urban contexts. Therefore, the study's focus on these interactions makes it highly relevant to the topic.


In [5]:
pip install openxyl

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement openxyl (from versions: none)
ERROR: No matching distribution found for openxyl

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [30]:
import openai
import pandas as pd

# Initialize OpenAI API
client.api_key ="sk-test"

# Read Excel file
file_path = "savedrecs.xls"  # Replace with the actual file path
data = pd.read_excel(file_path)

# Verify and match column names
expected_columns = ["Article Title", "Abstract", "DOI", "Times Cited, WoS Core"]
if not all(col in data.columns for col in expected_columns):
    raise ValueError("One or more expected columns are missing. Check the column names in the dataset.")

# Extract relevant columns
papers = data[expected_columns]

# Function to evaluate relevance
def evaluate_paper(title, abstract, keywords, citation_count):
    prompt = f"""
    You are an expert on human mobility and infectious disease modeling. Evaluate the following research paper based on the criteria below. Analyze the abstract and provided keywords carefully to extract specific disease, location information, meaningful keywords, and models mentioned. Provide structured and concise answers in JSON format.

    **Evaluation Criteria:**
    1. **Relevance Score (relevance_score):**
       - Rate the relevance of this paper to the topic of human mobility and its effects on infectious diseases on a scale of 1 to 10.
       - Provide a brief explanation for the score.

    2. **Publicly Available Dataset (is_dataset_public):**
        - Analyze the abstract and metadata for any mention of publicly available datasets.
        - Check the DOI and title on the internet and identify if it contains Publicly Available Dataset.
        - Extract dataset links (e.g., URLs) or dataset names.
        - If it says that the dataset cannot be provided, then write "not provided".

    3. **Citation Count (citation_count):**
       - The paper has been cited {citation_count} times according to the dataset.

    4. **Disease Focus (disease_focus):**
       - Analyze the abstract and identify the disease(s) this paper focuses on (e.g., "COVID-19", "malaria", "dengue", or other mentioned use cases or types of disease).
       - What disease is given as an example or a use case in the study?

    5. **Location Focus (location_focus):**
       - Analyze the abstract and determine the geographic or transport focus of the study (e.g., "city", "country", "plane", "train").
       - Provide specific names if mentioned, or indicate "global" if the scope is worldwide.

    6. **Keywords (keywords):**
       - Combine and analyze the abstract and provided keywords to extract 2-3 significant keywords summarizing the paper’s main topics.

    7. **Models Used (models):**
       - Extract and list the names of any models, algorithms, or methodologies mentioned in the abstract (e.g., "SEIR model", "machine learning", "gravity model").

    8. **Discussion Paper Recommendation (discussion_recommendation):**
       - Based on the above evaluation, comment on whether this paper should be included in a discussion paper about human mobility and infectious diseases. Provide a brief justification.

    **Paper Details:**
    Title: {title}
    Abstract: {abstract}
    Provided Keywords: {keywords}

    **Output Format:**
    Return a JSON object structured as follows:
    {{
      "relevance_score": "integer (1-10)",
      "is_dataset_public": "yes or no",
      "dataset_link": "string (link or dataset name, or 'not mentioned')",
      "citation_count": {citation_count},
      "disease_focus": "string ('disease name' or 'not specified')",
      "location_focus": "string ('location name' or 'global')",
      "keywords": ["keyword1", "keyword2", "keyword3"],
      "models": ["model1", "model2", "model3"],
      "discussion_recommendation": "string (recommendation with brief justification)"
    }}

    Example Response:
    {{
      "relevance_score": 9,
      "is_dataset_public": "yes",
      "dataset_link": "https://example.com/dataset1",
      "citation_count": {citation_count},
      "disease_focus": "malaria",
      "location_focus": "Bioko Island, Equatorial Guinea",
      "keywords": ["human mobility", "infectious diseases", "malaria transmission"],
      "models": ["gravity model", "SEIR model"],
      "discussion_recommendation": "Yes, this paper should be included because it directly examines human mobility's impact on malaria transmission and provides a publicly available dataset."
    }}
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in human mobility and infectious diseases."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error processing paper: {title}. Error: {e}")
        return None

# Process each paper
results = []
for index, row in papers.iterrows():  # Process only the first 2 papers
    title = row["Article Title"]
    abstract = row["Abstract"]
    doi = row["DOI"]
    citation_count = row["Times Cited, WoS Core"] if pd.notna(row["Times Cited, WoS Core"]) else "unknown"
    keywords = row.get("Keywords Plus", "None provided")  # Use 'Keywords Plus' or similar
    
    if pd.notna(title) and pd.notna(abstract):
        print(f"Evaluating paper: {title}")
        result = evaluate_paper(title, abstract, keywords, citation_count)
        if result:
            results.append({
                "Title": title,
                "DOI": doi,
                "Evaluation": result
            })

# Save results to a CSV file
output_file = "paper_relevance_scores.csv"
output_data = pd.DataFrame(results)
output_data.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")


Evaluating paper: Comparing metapopulation dynamics of infectious diseases under different models of human movement
Evaluating paper: Evaluating the impact of human flow on the spread of infectious diseases
Evaluating paper: Modeling Impact of Temperature and Human Movement on the Persistence of Dengue Disease
Evaluating paper: Human migration and infectious diseases
Evaluating paper: THE IMPORTANCE OF MOVEMENT IN ENVIRONMENTAL CHANGE AND INFECTIOUS DISEASE
Evaluating paper: Human mobility and the infectious disease transmission: a systematic review
Evaluating paper: Human mobility and the infectious disease transmission: a systematic review
Evaluating paper: Consequences of China's special send-down movement on infectious disease control in rural areas: a natural experiment
Evaluating paper: Contact structure, mobility, environmental impact and behaviour: the importance of social forces to infectious disease dynamics and disease ecology
Evaluating paper: Seasonal Population Movements 

In [ ]:
import json

# Function to parse the JSON-like Evaluation column
def parse_evaluation(row):
    try:
        # Extract and clean JSON-like data
        eval_json = row.replace("```json", "").replace("```", "").strip()
        return json.loads(eval_json)
    except Exception as e:
        return {"relevance_score": None, "is_data_available": None, "is_model_used": None}

# Apply the function to the Evaluation column
parsed_evaluations = data['Evaluation'].apply(parse_evaluation)

# Create separate columns for parsed JSON data
evaluation_df = pd.DataFrame(parsed_evaluations.tolist())

# Combine the original data with the parsed evaluation data
final_data = pd.concat([data.drop(columns=['Evaluation']), evaluation_df], axis=1)

# Save the cleaned and structured data to a new CSV file
output_path = '/mnt/data/cleaned_relevance_scores.csv'
final_data.to_csv(output_path, index=False)

# Display the cleaned data to the user
import ace_tools as tools; tools.display_dataframe_to_user(name="Cleaned Relevance Scores Dataset", dataframe=final_data)

output_path